In [0]:
dbutils.widgets.text("pipeline", "demo_pipeline", "pipeline name")
dbutils.widgets.text("entity", "orders", "entity to process")
dbutils.widgets.text("mode", "incremental", "full or incremental")
dbutils.widgets.text("target_schema", "workspace.default", "target schema")

pipeline = dbutils.widgets.get("pipeline")
entity = dbutils.widgets.get("entity")
mode = dbutils.widgets.get("mode").lower()
target_schema = dbutils.widgets.get("target_schema")
print(pipeline, entity, mode, target_schema)

In [0]:
bronze_tbl = f"{target_schema}.bronze_{entity}"
silver_tbl = f"{target_schema}.silver_{entity}"
watermark_tbl  = f"{target_schema}.etl_watermarks"
print("bronze:", bronze_tbl)
print("silver:", silver_tbl)
print("watermark:", watermark_tbl)

In [0]:
import pyspark.sql.functions as F

if mode == "incremental":
     wm = spark.sql(f"""
      SELECT watermark_ts
      FROM {watermark_tbl}
      WHERE pipeline='{pipeline}' AND entity='{entity}'
    """).first()[0]
else:
    wm = None
print(wm)


In [0]:
bronze = spark.table(bronze_tbl)
if mode == "incremental":
    df_in = bronze.filter(F.col("_ingested_at") > F.lit(wm))
else:
    df_in = bronze

print("input rows:", df_in.count())

In [0]:
spark.sql(f"""
INSERT INTO {target_schema}.etl_audit_results
SELECT
  '{pipeline}' as run_id,
  '{pipeline}' as pipeline,
  '{entity}' as entity,
  'job_start' as check_name,
  'PASS' as status,
  CAST(count(*) AS STRING) as metric_value,
  'Job started' as details,
  current_timestamp() as created_at
FROM {bronze_tbl}
""")

display(spark.sql(f"SELECT * FROM {target_schema}.etl_audit_results"))
